In [1]:
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow import keras
from tensorflow.python.keras import backend as K
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
import random
import pickle
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

2024-08-14 10:41:46.299119: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-14 10:41:46.572758: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-14 10:41:46.572815: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-14 10:41:46.627550: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-14 10:41:46.719369: I tensorflow/core/platform/cpu_feature_guar

In [2]:
drugs_cell_lines_ic50_df = pd.read_csv("..//data//drugs_cell_lines_ic50.csv")

In [3]:
drugs_cell_lines_ic50_df.head()

,drug_id,Cancer_Cell_Line,IC50
0,1001,ACH-002137,7.258918
1,1004,ACH-002137,-3.802467
2,1005,ACH-002137,4.146364
3,1006,ACH-002137,3.171367
4,1007,ACH-002137,-4.959442


In [4]:
pubchem_drugs_smiles_df = pd.read_csv('..//data//drugs_smile_strings.csv')

In [5]:
drugs_smiles_cell_lines_ic50_df = pd.merge(drugs_cell_lines_ic50_df, pubchem_drugs_smiles_df, 
                                             on = "drug_id")

In [6]:
drugs_smiles_cell_lines_ic50_df = drugs_smiles_cell_lines_ic50_df[["drug_id", "Cancer_Cell_Line", "Smiles", "IC50"]]

In [7]:
drugs_smiles_cell_lines_ic50_df.dtypes

drug_id               int64
Cancer_Cell_Line     object
Smiles               object
IC50                float64
dtype: object

In [8]:
drugs_smiles_cell_lines_ic50_df["drug_id"] = drugs_smiles_cell_lines_ic50_df["drug_id"].astype(object)

In [9]:
with open("..//data//drug_gcn_features.pickle", "rb") as f:
    dict_features = pickle.load(f)

In [10]:
with open("..//data//drug_gcn_normalized_adj_mats.pickle", "rb") as f:
    dict_normalized_adj_mats = pickle.load(f)

In [11]:
dualgcn_train = pd.read_csv("..//data//DualGCN_Embedding_train.csv")

In [12]:
dualgcn_test = pd.read_csv("..//data//DualGCN_Embedding_test.csv")

In [13]:
pubchem_to_drugs_df = pd.read_csv('..//data//1.Drug_listMon Jun 24 09_00_55 2019.csv')

In [14]:
pubchem_to_drugs_df.head()

,drug_id,Name,Synonyms,Targets,Target pathway,PubCHEM,Sample Size,Count
0,1242,(5Z)-7-Oxozeaenol,"5Z-7-Oxozeaenol, LL-Z1640-2",TAK1,"Other, kinases",9863776,945,266
1,179,5-Fluorouracil,5-FU,Antimetabolite (DNA & RNA),Other,3385,968,266
2,86,A-443654,KIN001-139,"AKT1, AKT2, AKT3",PI3K/MTOR signaling,10172943,425,266
3,55,A-770041,KIN001-111,"LCK, FYN","Other, kinases",9549184,426,266
4,1001,AICA Ribonucleotide,"AICAR, N1-(b-D-Ribofuranosyl)-5-aminoimidazole...",AMPK agonist,Metabolism,65110,872,266


In [15]:
pubchem_to_drugs_df = pubchem_to_drugs_df[["drug_id", "PubCHEM"]]

In [16]:
pubchem_to_drugs_df.dtypes

drug_id     int64
PubCHEM    object
dtype: object

In [17]:
pubchem_to_drugs_df["PubCHEM"] = [val if str(val).isdigit() else np.nan for val in pubchem_to_drugs_df["PubCHEM"] ]

In [18]:
pubchem_to_drugs_df = pubchem_to_drugs_df.dropna()

In [19]:
pubchem_to_drugs_df.dtypes

drug_id     int64
PubCHEM    object
dtype: object

In [20]:
pubchem_to_drugs_df["drug_id"] = pubchem_to_drugs_df["drug_id"].astype(str)

In [21]:
x_train, x_valid, y_train, y_valid = train_test_split(drugs_smiles_cell_lines_ic50_df.drop(["IC50"],axis = 1), drugs_smiles_cell_lines_ic50_df["IC50"].values, 
                                                     test_size = 0.20, random_state = 42)

In [22]:
dualgcn_train["Drug_ID"] = dualgcn_train["Drug_ID"].astype(str)

In [23]:
dualgcn_test["Drug_ID"] = dualgcn_test["Drug_ID"].astype(str)

In [24]:
pubchem_to_drugs_df.dtypes

drug_id    object
PubCHEM    object
dtype: object

In [25]:
dualgcn_train = pubchem_to_drugs_df.merge(dualgcn_train, left_on = ["PubCHEM"], right_on = ["Drug_ID"])

In [26]:
dualgcn_train = dualgcn_train[['Cell_Line', 'drug_id']]

In [27]:
dualgcn_test = pubchem_to_drugs_df.merge(dualgcn_test, left_on = ["PubCHEM"], right_on = ["Drug_ID"])

In [28]:
dualgcn_test = dualgcn_test[['Cell_Line', 'drug_id']]

In [29]:
dualgcn_train.dtypes

Cell_Line    object
drug_id      object
dtype: object

In [30]:
x_train.dtypes

drug_id             object
Cancer_Cell_Line    object
Smiles              object
dtype: object

In [31]:
x_train['drug_id'] = x_train['drug_id'].astype(str)

In [32]:
x_valid['drug_id'] = x_valid['drug_id'].astype(str)

In [33]:
x_train_valid_feats = pd.concat([x_train, x_valid], ignore_index = True)

In [34]:
y_train_valid = pd.concat([pd.DataFrame(y_train.reshape(-1,1)), pd.DataFrame(y_valid.reshape(-1,1))], ignore_index = True)

In [35]:
combo_train_valid = pd.concat([x_train_valid_feats, y_train_valid], axis = 1)

In [36]:
combo_train_valid.head()

,drug_id,Cancer_Cell_Line,Smiles,0
0,1030,ACH-000295,O=c1cc(-c2cccc3c2Sc2ccccc2S3)oc(N2CCOCC2)c1,3.143044
1,203,ACH-000292,Cc1ccc2nc(NCCN)c3ncc(C)n3c2c1,2.772620
2,1072,ACH-002273,NC(=O)C(CCC(F)(F)F)N(Cc1ccc(-c2ncon2)cc1F)S(=O...,3.809977
3,150,ACH-000631,CC(O)(CS(=O)(=O)c1ccc(F)cc1)C(=O)Nc1ccc(C#N)c(...,3.053030
4,282,ACH-000270,CCOc1cc2ncc(C#N)c(Nc3ccc(F)c(Cl)c3)c2cc1NC(=O)...,-0.367467


In [37]:
combo_train_valid.columns = ['drug_id', 'Cancer_Cell_Line', 'Smiles', 'IC50']

In [38]:
# filter x_train x _valid here
x_y_train = combo_train_valid.merge(dualgcn_train, left_on = ['Cancer_Cell_Line','drug_id'], right_on = [ 'Cell_Line','drug_id'])

In [39]:
x_y_train.head()

,drug_id,Cancer_Cell_Line,Smiles,IC50,Cell_Line
0,203,ACH-000292,Cc1ccc2nc(NCCN)c3ncc(C)n3c2c1,2.772620,ACH-000292
1,282,ACH-000270,CCOc1cc2ncc(C#N)c(Nc3ccc(F)c(Cl)c3)c2cc1NC(=O)...,-0.367467,ACH-000270
2,51,ACH-000087,Cc1nc(Nc2ncc(C(=O)Nc3c(C)cccc3Cl)s2)cc(N2CCN(C...,2.620152,ACH-000087
3,167,ACH-000948,NCC(=O)Nc1ccc(-n2nc(C(F)(F)F)cc2-c2ccc3c(ccc4c...,2.061684,ACH-000948
4,180,ACH-000971,CC=C(C)C(=O)OC1C(C)=C2C(C1OC(=O)CCCCCCC)C(C)(O...,-5.105745,ACH-000971


In [40]:
x_y_test = combo_train_valid.merge(dualgcn_test, left_on = ['Cancer_Cell_Line','drug_id'], right_on = [ 'Cell_Line','drug_id'])

In [41]:
x_train, x_valid, y_train, y_valid = x_y_train.drop(["IC50", 'Cell_Line'],axis = 1), x_y_test.drop(["IC50", 'Cell_Line'], axis = 1), x_y_train["IC50"].values, x_y_test["IC50"].values

In [42]:
np.mean(y_valid), np.std(y_valid)

(2.063824293081543, 2.8309603843576316)

In [43]:
# from deepcdr train
# (2.063824293081543, 2.8309603843576316)

In [44]:
train_gcn_feats = []
train_adj_list = []
for drug_id in x_train["drug_id"].values:
    train_gcn_feats.append(dict_features[drug_id])
    train_adj_list.append(dict_normalized_adj_mats[drug_id])

In [45]:
valid_gcn_feats = []
valid_adj_list = []
for drug_id in x_valid["drug_id"].values:
    valid_gcn_feats.append(dict_features[drug_id])
    valid_adj_list.append(dict_normalized_adj_mats[drug_id])

In [46]:
train_gcn_feats = np.array(train_gcn_feats).astype("float16")
valid_gcn_feats = np.array(valid_gcn_feats).astype("float16")

In [47]:
train_adj_list = np.array(train_adj_list).astype("float16")
valid_adj_list = np.array(valid_adj_list).astype("float16")

In [48]:
# load models
# omic models
cancer_copy_number_model = tf.keras.models.load_model("..//models//cancer_copy_number_model_no_norm_common")
cancer_cell_gen_expr_model = tf.keras.models.load_model("..//models//cancer_cell_gen_expr_model_no_norm_common")
cancer_cell_gen_methy_model = tf.keras.models.load_model("..//models//cancer_cell_gen_methy_model_no_norm")
cancer_cell_gen_mut_model = tf.keras.models.load_model("..//models//cancer_cell_gen_mut_model_no_norm")

2024-08-14 10:42:13.227080: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31141 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:06:00.0, compute capability: 7.0


In [49]:
# load models
# drug models
pubchem_drugs_rdkit_model = tf.keras.models.load_model("..//models//pubchem_drugs_rdkit_model_no_norm")

In [50]:
std = StandardScaler()

In [51]:
# extract drug features - does not seem like these are used in the network
drug_features_train = pubchem_drugs_rdkit_model(x_train["drug_id"].values).numpy().astype("float32")
drug_features_valid = pubchem_drugs_rdkit_model(x_valid["drug_id"].values).numpy().astype("float32")

# drug_features_train = pubchem_drugs_rdkit_model(x_train["drug_id"].values)
# drug_features_valid = pubchem_drugs_rdkit_model(x_valid["drug_id"].values)

In [52]:
np.isinf(drug_features_train).sum()

0

In [53]:
drug_features_train = std.fit_transform(drug_features_train)

In [54]:
drug_features_valid = std.transform(drug_features_valid)

In [55]:
# extract copy number features
omics_copy_number_train = cancer_copy_number_model(x_train["Cancer_Cell_Line"].values).numpy().astype("float16")
omics_copy_number_valid = cancer_copy_number_model(x_valid["Cancer_Cell_Line"].values).numpy().astype("float16")

In [56]:
# extract gen expr features
omics_gen_expr_train = cancer_cell_gen_expr_model(x_train["Cancer_Cell_Line"].values).numpy().astype("float16")
omics_gen_expr_valid = cancer_cell_gen_expr_model(x_valid["Cancer_Cell_Line"].values).numpy().astype("float16")

In [57]:
omics_gen_copy_number_gen_expr_train = np.concatenate([np.expand_dims(omics_copy_number_train, -1),
                                                      np.expand_dims(omics_gen_expr_train, -1)], axis = -1)

In [58]:
omics_gen_copy_number_gen_expr_valid = np.concatenate([np.expand_dims(omics_copy_number_valid, -1),
                                                      np.expand_dims(omics_gen_expr_valid, -1)], axis = -1)

In [59]:
# extract gen methylation features
omics_gen_methyl_train = cancer_cell_gen_methy_model(x_train["Cancer_Cell_Line"].values).numpy().astype("float16")
omics_gen_methyl_valid = cancer_cell_gen_methy_model(x_valid["Cancer_Cell_Line"].values).numpy().astype("float16")

In [60]:
# extract gen mutation features
with tf.device('/cpu:0'):
    omics_gen_mut_train = cancer_cell_gen_mut_model.predict(x_train["Cancer_Cell_Line"].values, verbose = 1, batch_size = 256).astype("float16")
    omics_gen_mut_valid = cancer_cell_gen_mut_model.predict(x_valid["Cancer_Cell_Line"].values, verbose = 1, batch_size = 256).astype("float16")

68/68 [==============================] - 0s 6ms/step


In [61]:
smile_strings_train = x_train["Smiles"].values.reshape(-1,1)
smile_strings_valid = x_valid["Smiles"].values.reshape(-1,1)

In [62]:
selected_info_common_cell_lines = "..//data//cellline_list.txt"
selected_info_common_genes = "..//data//gene_list.txt"

In [63]:
PPI_file = "..//data/PPI_network.txt"

In [64]:
with open(selected_info_common_cell_lines) as f:
    common_cell_lines = [item.strip() for item in f.readlines()]

In [65]:
with open("..//data//common_genes.pickle", "rb") as f:
    common_genes = pickle.load(f)

In [66]:
idx_dic={}
for index, item in enumerate(common_genes):
    idx_dic[item] = index

In [67]:
ppi_adj_info = [[] for item in common_genes] 

In [68]:
# will return for each gene what other gene is connected - PPIs
ppi_adj_info = [[] for item in common_genes] 
for line in open(PPI_file).readlines():
    gene1,gene2 = line.split('\t')[0],line.split('\t')[1]
    if (gene1 in common_genes) & (gene2 in common_genes):
        if idx_dic[gene1]<=idx_dic[gene2]:
            ppi_adj_info[idx_dic[gene1]].append(idx_dic[gene2])
            ppi_adj_info[idx_dic[gene2]].append(idx_dic[gene1])

In [69]:
def CelllineGraphAdjNorm(ppi_adj_info,common_genes = common_genes):
    # with open(selected_info_common_genes) as f:
    #     common_genes = [item.strip() for item in f.readlines()]
    nb_nodes = len(common_genes)
    adj_mat = np.zeros((nb_nodes,nb_nodes),dtype='float32')
    # print(adj_mat.shape)
    for i in range(len(ppi_adj_info)):
        # print(i)
        nodes = ppi_adj_info[i]
        for each in nodes:
            adj_mat[i,each] = 1

    # for checking if two sparse matrices are the same
    assert np.allclose(adj_mat,adj_mat.T)
    norm_adj = NormalizeAdj(adj_mat)
    return norm_adj 

In [70]:
def NormalizeAdj(adj):
    adj = adj + np.eye(adj.shape[0])
    d = sp.diags(np.power(np.array(adj.sum(1)), -0.5).flatten(), 0).toarray()
    a_norm = adj.dot(d).transpose().dot(d)
    return a_norm

In [71]:
import scipy.sparse as sp

In [72]:
ppi_adj = CelllineGraphAdjNorm(ppi_adj_info,common_genes)

In [73]:
ppi_adj = np.expand_dims(ppi_adj,0)

In [74]:
omics_gen_copy_number_gen_expr_train_new = (ppi_adj@omics_gen_copy_number_gen_expr_train)
omics_gen_copy_number_gen_expr_valid_new = (ppi_adj@omics_gen_copy_number_gen_expr_valid)

In [75]:
copy_number_train = omics_gen_copy_number_gen_expr_train_new[:,:,0:1]
copy_number_valid = omics_gen_copy_number_gen_expr_valid_new[:,:,0:1]

In [76]:
gene_expr_train = omics_gen_copy_number_gen_expr_train_new[:,:,1:2]
gene_expr_valid = omics_gen_copy_number_gen_expr_valid_new[:,:,1:2]

In [77]:
valid_items = [[ valid_gcn_feats, valid_adj_list,
                           copy_number_valid, gene_expr_valid,
                           omics_gen_methyl_valid, omics_gen_mut_valid], y_valid]

In [78]:
input_gcn_features = tf.keras.layers.Input(shape = (100, 75))
input_norm_adj_mat = tf.keras.layers.Input(shape = (100, 100))
mult_1 = tf.keras.layers.Dot(1)([input_norm_adj_mat, input_gcn_features])
dense_layer_gcn = tf.keras.layers.Dense(256, activation = "relu")
dense_out = dense_layer_gcn(mult_1)
dense_out = tf.keras.layers.BatchNormalization()(dense_out)
dense_out = tf.keras.layers.Dropout(0.1)(dense_out)
mult_2 = tf.keras.layers.Dot(1)([input_norm_adj_mat, dense_out])
dense_layer_gcn = tf.keras.layers.Dense(128, activation = "relu")
dense_out = dense_layer_gcn(mult_2)
dense_out = tf.keras.layers.BatchNormalization()(dense_out)
dense_out = tf.keras.layers.Dropout(0.1)(dense_out)

# dense_layer_gcn = tf.keras.layers.Dense(100, activation = "relu")
# mult_3 = tf.keras.layers.Dot(1)([input_norm_adj_mat, dense_out])
# dense_out = dense_layer_gcn(mult_3)
# dense_out = tf.keras.layers.BatchNormalization()(dense_out)
# dense_out = tf.keras.layers.Dropout(0.2)(dense_out)

dense_out = tf.keras.layers.GlobalAvgPool1D()(dense_out)

In [79]:
# here is the code for CNV and gene expression
dropout1 = 0.10
dropout2 = 0.20
# first add the CNV
input_cnv = tf.keras.layers.Input(shape = (omics_gen_expr_train.shape[1],1))
    
l1 = tf.keras.layers.Dense(32)(input_cnv)
l1 = tf.keras.layers.Dropout(dropout1)(l1)
l2 = tf.keras.layers.Dense(128)(l1)
l2 = tf.keras.layers.Dropout(dropout1)(l2)
    
dense_layer_gcn1 = tf.keras.layers.Dense(256, activation = "relu")
dense_out_cnv = dense_layer_gcn1(l2)
dense_out_cnv = tf.keras.layers.BatchNormalization()(dense_out_cnv)
dense_out_cnv = tf.keras.layers.Dropout(dropout1)(dense_out_cnv)
# mult_21 = tf.keras.layers.Dot(1)([const_input, dense_out1])
dense_layer_gcn1 = tf.keras.layers.Dense(256, activation = "relu")
dense_out_cnv = dense_layer_gcn1(dense_out_cnv)
dense_out_cnv = tf.keras.layers.BatchNormalization()(dense_out_cnv)
dense_out_cnv = tf.keras.layers.Dropout(dropout1)(dense_out_cnv)
dense_layer_gcn1 = tf.keras.layers.Dense(256, activation = "relu")
dense_out_cnv = dense_layer_gcn1(dense_out_cnv)
dense_out_cnv = tf.keras.layers.BatchNormalization()(dense_out_cnv)
dense_out_cnv = tf.keras.layers.Dropout(dropout1)(dense_out_cnv)
dense_layer_gcn1 = tf.keras.layers.Dense(256, activation = "relu")
dense_out_cnv = dense_layer_gcn1(dense_out_cnv)
dense_out_cnv = tf.keras.layers.BatchNormalization()(dense_out_cnv)
dense_out_cnv = tf.keras.layers.Dropout(dropout1)(dense_out_cnv)
dense_out_cnv = tf.keras.layers.GlobalAvgPool1D()(dense_out_cnv)

# now add the gene expr
input_gene_expr = tf.keras.layers.Input(shape = (omics_gen_expr_train.shape[1],1))
    
l11 = tf.keras.layers.Dense(32)(input_gene_expr)
l11 = tf.keras.layers.Dropout(dropout1)(l11)
l21 = tf.keras.layers.Dense(128)(l11)
l21 = tf.keras.layers.Dropout(dropout1)(l21)
    
dense_layer_gcn2 = tf.keras.layers.Dense(256, activation = "relu")
dense_out_expr = dense_layer_gcn2(l21)
dense_out_expr = tf.keras.layers.BatchNormalization()(dense_out_expr)
dense_out_expr = tf.keras.layers.Dropout(dropout1)(dense_out_expr)
# mult_21 = tf.keras.layers.Dot(1)([const_input, dense_out1])
dense_layer_gcn2 = tf.keras.layers.Dense(256, activation = "relu")
dense_out_expr = dense_layer_gcn2(dense_out_expr)
dense_out_expr = tf.keras.layers.BatchNormalization()(dense_out_expr)
dense_out_expr = tf.keras.layers.Dropout(dropout1)(dense_out_expr)
dense_layer_gcn2 = tf.keras.layers.Dense(256, activation = "relu")
dense_out_expr = dense_layer_gcn2(dense_out_expr)
dense_out_expr = tf.keras.layers.BatchNormalization()(dense_out_expr)
dense_out_expr = tf.keras.layers.Dropout(dropout1)(dense_out_expr )
dense_layer_gcn2 = tf.keras.layers.Dense(256, activation = "relu")
dense_out_expr = dense_layer_gcn2(dense_out_expr)
dense_out_expr = tf.keras.layers.BatchNormalization()(dense_out_expr)
dense_out_expr = tf.keras.layers.Dropout(dropout1)(dense_out_expr)
dense_out_expr = tf.keras.layers.GlobalAvgPool1D()(dense_out_expr)

In [80]:
all_omics = tf.keras.layers.Concatenate()([ dense_out_cnv, dense_out_expr, dense_out])

In [81]:
x = tf.keras.layers.Dense(256,activation = 'tanh')(all_omics)
x = tf.keras.layers.Dropout(0.3)(x, training = True)
x = tf.keras.layers.Dense(128,activation = 'tanh')(x)
x = tf.keras.layers.Dropout(0.2)(x, training = True)
x = tf.keras.layers.Dense(10,activation = 'tanh')(x)

In [82]:
final_out_layer = tf.keras.layers.Dense(1)

In [83]:
final_out = final_out_layer(x)

In [84]:
simplegcn = tf.keras.models.Model([input_gcn_features, input_norm_adj_mat, input_cnv, input_gene_expr], final_out)

In [85]:
simplegcn.compile(loss = tf.keras.losses.MeanSquaredError(), 
                    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001), 
                    metrics = [tf.keras.metrics.RootMeanSquaredError()])

In [86]:
# import the existing trained model
simpleGCN_original_model = tf.keras.models.load_model("..//models//dualgcn_trained_on_domain")

In [87]:
simplegcn.set_weights(simpleGCN_original_model.get_weights())

In [88]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [89]:
chunks = np.array_split(list(range(0,valid_gcn_feats.shape[0])), 3)

In [90]:
test_index = 2

In [91]:
train_items = list(chunks[test_index])

In [92]:
test_items = list(set(list(range(0,valid_gcn_feats.shape[0]))).difference(train_items))

In [93]:
# First the data - let's use a train set of 6k - then the rest will be test here (from the remaining values, use 3k for training the stacker, 1k for validation of it, and the rest ~7k as the final test data). 
# drug feats
new_train_gcn_feats = valid_gcn_feats[train_items, :,:]
new_test_gcn_feats = valid_gcn_feats[test_items, :,:]
print(new_train_gcn_feats.shape, new_test_gcn_feats.shape)

# drug adj info
new_train_adj_list = valid_adj_list[train_items, :,:]
new_test_adj_list = valid_adj_list[test_items, :,:]
print(new_train_adj_list.shape, new_test_adj_list.shape)

# cnv
new_omics_cna_train = copy_number_valid[train_items, :]
new_omics_cna_test = copy_number_valid[test_items,:]
print(new_omics_cna_train.shape, new_omics_cna_test.shape)

# expression
new_omics_expr_train = gene_expr_valid[train_items, :]
new_omics_expr_test = gene_expr_valid[test_items,:]
print(new_omics_cna_train.shape, new_omics_cna_test.shape)

# y
new_y_train = y_valid[train_items,]
new_y_test = y_valid[test_items,]
print(new_y_train.shape, new_y_test.shape)

(5772, 100, 75) (11544, 100, 75)
(5772, 100, 100) (11544, 100, 100)
(5772, 691, 1) (11544, 691, 1)
(5772, 691, 1) (11544, 691, 1)
(5772,) (11544,)


In [94]:
# from deepcdr fold 0
# (2.069332453915454, 2.8460217894549977, 2.0610702126645877, 2.8233955229121412)

In [95]:
np.mean(new_y_train), np.std(new_y_train), np.mean(new_y_test), np.std(new_y_test)

(2.094244087144837, 2.8136855131635183, 2.048614396049896, 2.8394362011334575)

In [96]:
%%time
history = simplegcn.fit([new_train_gcn_feats, new_train_adj_list, new_omics_cna_train, new_omics_expr_train], new_y_train, 
                         
          batch_size = 512, epochs = 1000, verbose = 1,
                         
          validation_split = 0.2,
                         

        callbacks = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 20, restore_best_weights=True,
                                                       mode = "min"), 
         validation_batch_size = 512, shuffle = True)

Epoch 1/1000


2024-08-14 10:43:05.178925: I external/local_xla/xla/service/service.cc:168] XLA service 0x14f30464ce20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-08-14 10:43:05.178984: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100S-PCIE-32GB, Compute Capability 7.0
2024-08-14 10:43:05.215560: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-14 10:43:05.301772: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
I0000 00:00:1723650185.508365 1494878 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


10/10 [==============================] - 13s 343ms/step - loss: 1.8506 - root_mean_squared_error: 1.3604 - val_loss: 1.8351 - val_root_mean_squared_error: 1.3547
Epoch 2/1000
10/10 [==============================] - 3s 259ms/step - loss: 1.8155 - root_mean_squared_error: 1.3474 - val_loss: 1.8407 - val_root_mean_squared_error: 1.3567
Epoch 3/1000
10/10 [==============================] - 3s 261ms/step - loss: 1.8335 - root_mean_squared_error: 1.3541 - val_loss: 1.8846 - val_root_mean_squared_error: 1.3728
Epoch 4/1000
10/10 [==============================] - 3s 261ms/step - loss: 1.8259 - root_mean_squared_error: 1.3513 - val_loss: 1.8264 - val_root_mean_squared_error: 1.3514
Epoch 5/1000
10/10 [==============================] - 3s 261ms/step - loss: 1.8267 - root_mean_squared_error: 1.3515 - val_loss: 1.8258 - val_root_mean_squared_error: 1.3512
Epoch 6/1000
10/10 [==============================] - 3s 261ms/step - loss: 1.8194 - root_mean_squared_error: 1.3489 - val_loss: 1.8067 - val_

In [97]:
simplegcn.save("..//models//dualgcn_complete_fine_tuned_split_" + str(test_index))

INFO:tensorflow:Assets written to: ..//models//dualgcn_complete_fine_tuned_split_2/assets


INFO:tensorflow:Assets written to: ..//models//dualgcn_complete_fine_tuned_split_2/assets


In [98]:
reps = 25

In [99]:
%%time
catch_preds = np.zeros((new_test_gcn_feats.shape[0],reps))
for i in range(0,reps):
    print(i)
    preds = simplegcn.predict([ new_test_gcn_feats, new_test_adj_list, new_omics_cna_test, new_omics_expr_test], 
                              batch_size = 1024, verbose = 0)
    catch_preds[:,i] = preds[:,0]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
CPU times: user 48.7 s, sys: 7.85 s, total: 56.6 s
Wall time: 1min 8s


In [100]:
preds_data = np.hstack((new_y_test.reshape(-1,1), catch_preds))

In [101]:
np.save("..//data//dualgcn_complete_preds_val_and_test_" +str(test_index) + ".npy", preds_data)